# EDGARTools Playground (Rav)

Interactive notebook to explore:

- Company metadata
- Latest 10-K / 10-Q
- Balance sheet / income statement / cash flow as DataFrames
- Insider Form 4 filings

Requires:
- `edgartools` installed (`pip install edgartools`)
- A real email for SEC identity


In [54]:
# If edgartools is not installed in this environment, uncomment:
# !pip install edgartools

from edgar import Company, set_identity
from edgar.xbrl.xbrl import XBRL
import pandas as pd

# 👇 IMPORTANT: use your real email (SEC user-agent requirement)
set_identity("kamboj.r@gmail.com")

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)


In [55]:
def normalize_statement(stmt):
    """
    Take whatever edgartools returns for a statement
    and turn it into a pandas DataFrame.

    Handles:
    - functions that must be called
    - objects with .get_dataframe()
    - objects with .to_pandas() / .to_dataframe()
    - a raw pandas DataFrame
    """
    # If it's callable (function/method), call it
    if callable(stmt):
        stmt = stmt()

    # Try common methods
    for attr in ("get_dataframe", "to_pandas", "to_dataframe"):
        if hasattr(stmt, attr):
            return getattr(stmt, attr)()

    # If it's already a DataFrame, just return it
    if isinstance(stmt, pd.DataFrame):
        return stmt

    # Last resort: show what we got
    raise TypeError(f"Unexpected statement type: {type(stmt)}")


In [56]:
def get_latest_filing_obj(cik_or_ticker: str, form: str = "10-K"):
    """
    Returns the latest filing object (e.g. TenK / TenQ) for a given ticker/CIK.
    """
    company = Company(cik_or_ticker)
    filings = company.get_filings(form=form)
    latest = filings.latest(1).obj()
    return latest


In [57]:
# You can use ticker "AAPL" or CIK "0000320193"
company = Company("SHOP")

company


╭───────────────────────────────────────────── 🏢 SHOPIFY INC. SHOP ──────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ ╭───────────────────────────────────────────────── 📋 Entity ─────────────────────────────────────────────────╮ │
│ │                                                                                                             │ │
│ │   CIK   1594805   Type   Operating   ○                                                                      │ │
│ │                                                                                                             │ │
│ │                                                                                                             │ │
│ │   Category                  Industry                              Fisc

In [59]:
#Single, simple filing for 10-K.

from edgar import Company
from edgar.xbrl.xbrl import XBRL

# Get a company's latest 10-K filing
company = Company('MSFT')
filing = company.latest("10-K")

# Parse XBRL data
xb = filing.xbrl()

# Access statements through the user-friendly API
statements = xb.statements

# Display financial statements
balance_sheet = statements.balance_sheet()
income_statement = statements.income_statement()
cash_flow = statements.cashflow_statement()

#Convert each statement to dataframes
balance_sheet_df = statements.balance_sheet()
income_stmt_df = statements.income_statement()
cash_flow_df = statements.cashflow_statement()

print(income_statement)
print(balance_sheet)
print(cash_flow)



                                      Consolidated Statement of Income                                       
                                                 Year Ended                                                  
                               (In millions, except shares and per share data)                               
                                                                                                             
                                                                 Jun 30, 2025   Jun 30, 2024   Jun 30, 2023  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────── 
    Product and Service                                                                                      
      Product and Service                                                                                    
          Product                                                                                            
          

                                        Consolidated Balance Sheets                                        
                                   As of June 30, 2025 and June 30, 2024                                   
                              (In millions, except shares and per share data)                              
                                                                                                           
                                                                              Jun 30, 2025   Jun 30, 2024  
 ───────────────────────────────────────────────────────────────────────────────────────────────────────── 
  Assets                                                                                                   
    Current assets:                                                                                        
        Cash and Cash Equivalents:                                                 $30,242        $18,315  
          Debt Securities - 

                                                         Consolidated Statement of Cash Flows                                                         
                                                                      Year Ended                                                                      
                                                   (In millions, except shares and per share data)                                                    
                                                                                                                                                      
                                                                                                          Jun 30, 2025   Jun 30, 2024   Jun 30, 2023  
 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  Operations                                                                                  

In [46]:
#Single, simple filing for 10-Q.

from edgar import Company
from edgar.xbrl.xbrl import XBRL

# Get a company's latest 10-K filing
company = Company('META')
filing = company.latest("10-Q")

# Parse XBRL data
xb = filing.xbrl()

# Access statements through the user-friendly API
statements = xb.statements

# Display financial statements
balance_sheet = statements.balance_sheet()
income_statement = statements.income_statement()
cash_flow = statements.cashflow_statement()

#Convert each statement to dataframes
balance_sheet_df = statements.balance_sheet()
income_stmt_df = statements.income_statement()
cash_flow_df = statements.cashflow_statement()

print(income_statement)
print(balance_sheet)
print(cash_flow)

                                                        Consolidated Statement of Income                                                         
                                                           Three and Nine Months Ended                                                           
                                                 (In millions, except shares and per share data)                                                 
                                                                                                                                                 
                                                                Sep 30, 2025 (Q3)   Sep 30, 2024 (Q3)   Sep 30, 2025 (YTD)   Sep 30, 2024 (YTD)  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
    Contract Revenue:                                                     $51,242             $40,589             $141,073  

                                      Consolidated Balance Sheets                                       
                             As of September 30, 2025 and December 31, 2024                             
                            (In millions, except shares and per share data)                             
                                                                                                        
                                                                           Sep 30, 2025   Dec 31, 2024  
 ────────────────────────────────────────────────────────────────────────────────────────────────────── 
  Assets                                                                                                
    Current assets:                                                                                     
        Cash and Cash Equivalents                                               $10,187        $43,889  
        Current Marketable Securities:                 

                                                         Consolidated Statement of Cash Flows                                                         
                                                                  Nine Months Ended                                                                   
                                                   (In millions, except shares and per share data)                                                    
                                                                                                                                                      
                                                                                                             Sep 30, 2025 (YTD)   Sep 30, 2024 (YTD)  
 ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  Cash flows from operating activities                                                        

In [53]:
import pandas as pd
from edgar.xbrl.analysis.financial_ratio_analyzer import FinancialRatioAnalyzer

# simple toy data
bs = pd.DataFrame([
    {"period": "2024Q1", "current_assets": 200.0, "current_liabilities": 100.0, "total_assets": 1000.0},
    {"period": "2024Q2", "current_assets": 250.0, "current_liabilities": 125.0, "total_assets": 1100.0},
])
is_stmt = pd.DataFrame([
    {"period": "2024Q1", "net_income": 50.0},
    {"period": "2024Q2", "net_income": 60.0},
])

analyzer = FinancialRatioAnalyzer(balance_sheet_df=bs, income_stmt_df=is_stmt)
current_ratios = analyzer.calculate_current_ratio()
roa_ratios = analyzer.calculate_return_on_assets()

for r in current_ratios:
    print(f"{r.name} ({r.period}): {r.result:.2f}")
for r in roa_ratios:
    print(f"{r.name} ({r.period}): {r.result:.4f}")

Current Ratio (2024Q1): 2.00
Current Ratio (2024Q2): 2.00
Return on Assets (2024Q1): 0.0500
Return on Assets (2024Q2): 0.0545
